In [3]:
!pip uninstall -y gcsfs fsspec

Found existing installation: gcsfs 2025.3.2
Uninstalling gcsfs-2025.3.2:
  Successfully uninstalled gcsfs-2025.3.2
Found existing installation: fsspec 2025.3.2
Uninstalling fsspec-2025.3.2:
  Successfully uninstalled fsspec-2025.3.2


In [2]:
!pip uninstall -y transformers datasets fsspec gcsfs
!pip install transformers==4.44.2 sentence-transformers==4.1.0 datasets==2.13.2 fsspec==2024.3.1 gcsfs==2024.3.1
# !pip uninstall -y transformers datasets fsspec gcsfs
# !pip install transformers==4.32.0 datasets==2.19.1 fsspec==2024.3.1 gcsfs==2024.3.1
!pip install -q seqeval accelerate

# !pip install transformers==4.41.2 datasets==2.19.1

# # !pip install -U transformers==4.41.2 datasets==2.19.1
# # !pip install -U fsspec==2024.3.1 gcsfs==2024.3.1

Found existing installation: transformers 4.44.2
Uninstalling transformers-4.44.2:
  Successfully uninstalled transformers-4.44.2
Found existing installation: datasets 2.13.2
Uninstalling datasets-2.13.2:
  Successfully uninstalled datasets-2.13.2
Found existing installation: fsspec 2024.3.1
Uninstalling fsspec-2024.3.1:
  Successfully uninstalled fsspec-2024.3.1
Found existing installation: gcsfs 2024.3.1
Uninstalling gcsfs-2024.3.1:
  Successfully uninstalled gcsfs-2024.3.1
  Using cached transformers-4.44.2-py3-none-any.whl.metadata (43 kB)
  Using cached datasets-2.13.2-py3-none-any.whl.metadata (20 kB)
  Using cached fsspec-2024.3.1-py3-none-any.whl.metadata (6.8 kB)
  Using cached gcsfs-2024.3.1-py2.py3-none-any.whl.metadata (1.6 kB)
Using cached transformers-4.44.2-py3-none-any.whl (9.5 MB)
Using cached datasets-2.13.2-py3-none-any.whl (512 kB)
Using cached fsspec-2024.3.1-py3-none-any.whl (171 kB)
Using cached gcsfs-2024.3.1-py2.py3-none-any.whl (34 kB)


In [3]:
from datasets import Dataset
import pandas as pd

from transformers import AutoTokenizer

from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer
import numpy as np
from seqeval.metrics import classification_report, f1_score

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

### Load and Label CoNLL Dataset

In [4]:
def parse_conll(filepath):
    sentences, labels = [], []
    with open(filepath, encoding='utf-8') as file:
        sentence, label = [], []
        for line in file:
            line = line.strip()
            if not line:
                if sentence:
                    sentences.append(sentence)
                    labels.append(label)
                    sentence, label = [], []
            else:
                splits = line.split()
                sentence.append(splits[0])
                label.append(splits[1])
    return sentences, labels

sentences, ner_labels = parse_conll("/content/drive/MyDrive/10-Academy/week4/amharic-ecommerce-ner/data/amharic_ner_conll.txt")
# Prepare dataset
data = [{"tokens": s, "ner_tags": l} for s, l in zip(sentences, ner_labels)]
dataset = Dataset.from_list(data)
dataset = dataset.train_test_split(test_size=0.2)


### Preprocessing and Tokenization

In [5]:
model_checkpoint = "Davlan/xlm-roberta-base-ner-hrl"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
label_list = sorted(list(set(tag for seq in ner_labels for tag in seq)))
label_to_id = {label: idx for idx, label in enumerate(label_list)}
id_to_label = {idx: label for label, idx in label_to_id.items()}

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py:1601: Future

In [10]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(examples["tokens"],
                                 truncation=True,
                                 padding=True,
                                 is_split_into_words=True,
                                 return_tensors=None)
    labels = []
    for i, label in enumerate(examples["ner_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            if word_idx is None:
                label_ids.append(-100)
            elif word_idx != previous_word_idx:
                label_ids.append(label_to_id[label[word_idx]])
            else:
                label_ids.append(label_to_id[label[word_idx]] if label[word_idx].startswith("I-") else label_to_id[label[word_idx]])
            previous_word_idx = word_idx
        labels.append(label_ids)
    tokenized_inputs["labels"] = labels
    return tokenized_inputs

tokenized_datasets = dataset.map(tokenize_and_align_labels, batched=True)

Map:   0%|          | 0/9 [00:00<?, ? examples/s]

Map:   0%|          | 0/3 [00:00<?, ? examples/s]

### Model and Training Arguments

In [12]:
model = AutoModelForTokenClassification.from_pretrained(
    model_checkpoint,
    num_labels=len(label_list),
    ignore_mismatched_sizes=True
    )

# from transformers import TrainingArguments

args = TrainingArguments(
    output_dir="/content/drive/MyDrive/10-Academy/week4/amharic-ecommerce-ner/results",
    evaluation_strategy="epoch",  # works now
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10,
    save_strategy="epoch"
)


Some weights of XLMRobertaForTokenClassification were not initialized from the model checkpoint at Davlan/xlm-roberta-base-ner-hrl and are newly initialized because the shapes did not match:
- classifier.bias: found shape torch.Size([9]) in the checkpoint and torch.Size([7]) in the model instantiated
- classifier.weight: found shape torch.Size([9, 768]) in the checkpoint and torch.Size([7, 768]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


### Define Metrics and Trainer

In [13]:
def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    true_labels = [[id_to_label[l] for l in label if l != -100] for label in labels]
    true_preds = [
        [id_to_label[pred] for pred, label in zip(prediction, label_row) if label != -100]
        for prediction, label_row in zip(predictions, labels)
    ]

    return {
        "f1": f1_score(true_labels, true_preds),
        "report": classification_report(true_labels, true_preds)
    }

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [14]:
trainer.train()

Epoch,Training Loss,Validation Loss,F1,Report
1,No log,1.720654,0.064516,precision recall f1-score support LOC 0.00 0.00 0.00 12 PRICE 0.00 0.00 0.00 4 Product 0.14 0.17 0.15 6 micro avg 0.11 0.05 0.06 22 macro avg 0.05 0.06 0.05 22 weighted avg 0.04 0.05 0.04 22
2,No log,1.623062,0.083333,precision recall f1-score support LOC 0.00 0.00 0.00 12 PRICE 0.00 0.00 0.00 4 Product 1.00 0.17 0.29 6 micro avg 0.50 0.05 0.08 22 macro avg 0.33 0.06 0.10 22 weighted avg 0.27 0.05 0.08 22
3,No log,1.583908,0.086957,precision recall f1-score support LOC 0.00 0.00 0.00 12 PRICE 0.00 0.00 0.00 4 Product 1.00 0.17 0.29 6 micro avg 1.00 0.05 0.09 22 macro avg 0.33 0.06 0.10 22 weighted avg 0.27 0.05 0.08 22


/usr/local/lib/python3.11/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
Trainer is attempting to log a value of "              precision    recall  f1-score   support

         LOC       0.00      0.00      0.00        12
       PRICE       0.00      0.00      0.00         4
     Product       0.14      0.17      0.15         6

   micro avg       0.11      0.05      0.06        22
   macro avg       0.05      0.06      0.05        22
weighted avg       0.04      0.05      0.04        22
" of type <class 'str'> for key "eval/report" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
/usr/local/lib/python3.11/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined

TrainOutput(global_step=3, training_loss=1.7968532244364421, metrics={'train_runtime': 117.8354, 'train_samples_per_second': 0.229, 'train_steps_per_second': 0.025, 'total_flos': 661437314208.0, 'train_loss': 1.7968532244364421, 'epoch': 3.0})

In [15]:
trainer.save_model("/content/drive/MyDrive/10-Academy/week4/amharic-ecommerce-ner/amharic-ner-model")
tokenizer.save_pretrained("/content/drive/MyDrive/10-Academy/week4/amharic-ecommerce-ner/amharic-ner-pretrained-model")

('/content/drive/MyDrive/10-Academy/week4/amharic-ecommerce-ner/amharic-ner-pretrained-model/tokenizer_config.json',
 '/content/drive/MyDrive/10-Academy/week4/amharic-ecommerce-ner/amharic-ner-pretrained-model/special_tokens_map.json',
 '/content/drive/MyDrive/10-Academy/week4/amharic-ecommerce-ner/amharic-ner-pretrained-model/sentencepiece.bpe.model',
 '/content/drive/MyDrive/10-Academy/week4/amharic-ecommerce-ner/amharic-ner-pretrained-model/added_tokens.json',
 '/content/drive/MyDrive/10-Academy/week4/amharic-ecommerce-ner/amharic-ner-pretrained-model/tokenizer.json')

In [16]:
trainer.evaluate()

/usr/local/lib/python3.11/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
Trainer is attempting to log a value of "              precision    recall  f1-score   support

         LOC       0.00      0.00      0.00        12
       PRICE       0.00      0.00      0.00         4
     Product       1.00      0.17      0.29         6

   micro avg       1.00      0.05      0.09        22
   macro avg       0.33      0.06      0.10        22
weighted avg       0.27      0.05      0.08        22
" of type <class 'str'> for key "eval/report" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.


{'eval_loss': 1.583907961845398,
 'eval_f1': 0.08695652173913045,
 'eval_report': '              precision    recall  f1-score   support\n\n         LOC       0.00      0.00      0.00        12\n       PRICE       0.00      0.00      0.00         4\n     Product       1.00      0.17      0.29         6\n\n   micro avg       1.00      0.05      0.09        22\n   macro avg       0.33      0.06      0.10        22\nweighted avg       0.27      0.05      0.08        22\n',
 'eval_runtime': 0.0391,
 'eval_samples_per_second': 76.637,
 'eval_steps_per_second': 25.546,
 'epoch': 3.0}

In [20]:
!cp ./amharic_ner_train.ipynb /content/drive/MyDrive/10-Academy/week4/amharic-ecommerce-ner/

cp: cannot stat './amharic_ner_train.ipynb': No such file or directory


In [21]:
!git config --global user.name "EmebetF"
!git config --global user.email "emebet1298@gmail.com"


In [22]:
%cd /content/
!git clone https://github.com/EmebetF/amharic-ecommerce-ner.git
%cd amharic-ecommerce-ner


/content
Cloning into 'amharic-ecommerce-ner'...
remote: Enumerating objects: 14, done.
remote: Counting objects: 100% (14/14), done.
remote: Compressing objects: 100% (11/11), done.
remote: Total 14 (delta 0), reused 10 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (14/14), 5.76 KiB | 2.88 MiB/s, done.
/content/amharic-ecommerce-ner


In [23]:
!git checkout -b task-3

Switched to a new branch 'task-3'


In [ ]:
!cp /content/drive/MyDrive/10-Academy/week4/amharic-ecommerce-ner/notebooks/amharic_ner_train.ipynb .
!cp -r /content/drive/MyDrive/10-Academy/week4/amharic-ecommerce-ner/ ./model_output
